# Module 7 - Programming Assignment

## Directions

1. Change the name of this file to be your JHED id as in `jsmith299.ipynb`. Because sure you use your JHED ID (it's made out of your name and not your student id which is just letters and numbers).
2. Make sure the notebook you submit is cleanly and fully executed. I do not grade unexecuted notebooks.
3. Submit your notebook back in Blackboard where you downloaded this file.

*Provide the output **exactly** as requested*

# Unification

This is actually Part I of a two part assignment. In a later module, you'll implement a Forward Planner. In order to do that, however, you need to have a unifier. It is important to note that you *only* need to implement a unifier. Although the module talked about resolution, you do not need to implement anything like "standardizing apart". From the unifier's point of view, that should already have been done.

Unification is simply the *syntactic* balancing of expressions. There are only 3 kinds of expressions: constants, lists and (logic) variables. Constants and lists are only equal to each other if they're exactly the same thing or can be made to be the same thing by *binding* a value to a variable.

It really is that simple...expressions must be literally the same (identical) except if one or the other (or both) has a variable in that "spot".

## S-Expressions

With that out of the way, we need a language with which to express our constants, variables and predicates and that language will be based on s-expressions.

**constants** - There are two types of constants, values and predicates. Values should start with an uppercase letter. Fred is a constant value, so is Barney and Food. Predicates are named using lowercase letters. loves is a predicate and so is hates. This is only a convention. Secret: your code does not need to treat these two types of constants differently.

**variables** - these are named using lowercase letters but always start with a question mark. ?x is a variable and so is ?yum. This is not a convention.

**expressions (lists)** - these use the S-expression syntax a la LISP. (loves Fred Wilma) is an expression as is (friend-of Barney Fred) and (loves ?x ?y).

## Parsing

These functions are already included in the starter .py file.

In [211]:
import tokenize
from io import StringIO

This uses the above libraries to build a Lisp structure based on atoms. It is adapted from [simple iterator parser](http://effbot.org/zone/simple-iterator-parser.htm). The first function is the `atom` function.

In [212]:
def atom( next, token):
    if token[ 1] == '(':
        out = []
        token = next()
        while token[ 1] != ')':
            out.append( atom( next, token))
            token = next()
            if token[ 1] == ' ':
                token = next()
        return out
    elif token[ 1] == '?':
        token = next()
        return "?" + token[ 1]
    else:
        return token[ 1]

The next function is the actual `parse` function:

In [213]:
def parse(exp):
    src = StringIO(exp).readline
    tokens = tokenize.generate_tokens(src)
    return atom(tokens.__next__, tokens.__next__())

**Note** there was a change between 2.7 and 3.0 that "hid" the next() function in the tokenizer.

From a Python perspective, we want to turn something like "(loves Fred ?x)" to ["loves" "Fred" "?x"] and then work with the second representation as a list of strings. The strings then have the syntactic meaning we gave them previously.

In [214]:
parse("Fred")

'Fred'

In [215]:
parse( "?x")

'?x'

In [216]:
parse( "(loves Fred ?x)")

['loves', 'Fred', '?x']

In [217]:
parse( "(father_of Barney (son_of Barney))")

['father_of', 'Barney', ['son_of', 'Barney']]

## Unifier

Now that that's out of the way, here is the imperative pseudocode for unification. This is a classic recursive program with a number of base cases. Students for some reason don't like it, try the algorithm in the book, can't get it to work and then come back to this pseudocode.

Work through the algorithm by hand with your Self-Check examples if you need to but I'd suggest sticking with this implementation. It does work.

Here is imperative pseudocode for the algorithm:

```
def unification( exp1, exp2):
    # base cases
    if exp1 and exp2 are constants or the empty list:
        if exp1 = exp2 then return {}
        else return FAIL
    if exp1 is a variable:
        if exp1 occurs in exp2 then return FAIL
        else return {exp1/exp2}
    if exp2 is a variable:
        if exp2 occurs in exp1 then return FAIL
        else return {exp2/exp1}

    # inductive step
    first1 = first element of exp1
    first2 = first element of exp2
    result1 = unification( first1, first2)
    if result1 = FAIL then return FAIL
    apply result1 to rest of exp1 and exp2
    result2 = unification( rest of exp1, rest of exp2)
    if result2 = FAIL then return FAIL
    return composition of result1 and result2
```

`unification` can return...

1. `None` (if unification completely fails)
2. `{}` (the empty substitution list) or 
3. a substitution list that has variables as keys and substituted values as values, like {"?x": "Fred"}. 

Note that the middle case sometimes confuses people..."Sam" unifying with "Sam" is not a failure so you return {} because there were no variables so there were no substitutions. You do not need to further resolve variables. If a variable resolves to an expression that contains a variable, you don't need to do the substition.

If you think of a typical database table, there is a column, row and value. This Tuple is a *relation* and in some uses of unification, the "thing" in the first spot..."love" above is called the relation. If you have a table of users with user_id, username and the value then the relation is:

`(login ?user_id ?username)`

*most* of the time, the relation name is specified. But it's not impossible for the relation name to be represented by a variable:

`(?relation 12345 "smooth_operator")`

Your code should handle this case (the pseudocode does handle this case so all  you have to do is not futz with it).

Our type system is very simple. We can get by with just a few boolean functions. The first tests to see if an expression is a variable.

In [218]:
def is_variable( exp):
    return isinstance( exp, str) and exp[ 0] == "?"

In [219]:
is_variable( "Fred")

False

In [220]:
is_variable( "?fred")

True

The second tests to see if an expression is a constant:

In [221]:
def is_constant( exp):
    return isinstance( exp, str) and not is_variable( exp)

In [222]:
is_constant( "Fred")

True

In [223]:
is_constant( "?fred")

False

In [224]:
is_constant( ["loves", "Fred", "?wife"])

False

It might also be useful to know that:

<code>
type( "a")
&lt;type 'str'>
type( "a") == str
True
type( "a") == list
False
type( ["a"]) == list
True
</code>


You need to write the `unification` function described above. It should work with two expressions of the type returned by `parse`. See `unify` for how it will be called. It should return the result of unification for the two expressions as detailed above and in the book. It does not have to make all the necessary substitions (for example, if ?y is bound to ?x and 1 is bound to ?y, ?x doesn't have to be replaced everywhere with 1. It's enough to return {"?x":"?y", "?y":1}. For an actual application, you would need to fix this!)

-----

### <a id="denester_replacer"></a> denester_replacer

Formal Parameters:
**l**: A string or list of strings to be replaced by a variable

**old**: The string (the variable) that should be replaced

**new**: What the variable should be replaced with (usually a constant)

**modifies**  **l** if **l** is a list, and if it contains a string that should be replaced.

**returns** **l**, or **new**, depending on if **l** should be replaced with **new** or not

This is the *apply result1 to rest of exp1 and exp2* part of the pseudocode.  It recursively denests **l** until **l** is a string, then replaces the string, if appropriate.

In [225]:
def denester_replacer(l,old,new):
    if type(l) == str:
        if l == old:
            return new
        else:
            return l
    else:
        for i in range(len(l)):
            l[i] = denester_replacer(l[i],old,new)
        return l

In [226]:
old = "x"
new = "y"
l1 = "x"
assert(denester_replacer(l1,old,new)==new)
l2 = ["x","y","z"]
assert(denester_replacer(l2,old,new)==["y","y","z"])
assert(l2!=["x","y","z"])
l3 = ["x",["x","z"]]
assert(denester_replacer(l3,old,new)==["y",["y","z"]])
assert(l3!=["x",["x","z"]])
l4 = ["w","z"]
assert(denester_replacer(l4,old,new)==["w","z"])
assert(l4 == ["w","z"])

### <a id="is_inside"></a> is_inside

Formal Parameters:
**var**: A string (variable)

**lst**: The list.

**returns** True or False, depending on whether **var** is in **lst**

This is the *if exp1 occurs in exp2 then return FAIL* part of the pseudocode, and the *Occurrence Check* in the textbook.  It recursively denests **lst**, checking if **var** is in the list or any nested list.

In [227]:
def is_inside(var,lst):
    inside = False
    if type(lst) == list:
        for x in lst:
            if var==x:
                return True
            inside = inside or is_inside(var,x)
    return inside

In [228]:
x = 1
l = [1,2,3]
assert(is_inside(x,l))
l2 = [2,[3,[1]]]
assert(is_inside(x,l2))
l3 = [2,3,4]
assert(not is_inside(x,l3))

### <a id="unification"></a> unification

Formal Parameters:
**list_expression1**: A list, representing a sentence in logic

**list_expression2**: A list, representing a sentence in logic

**returns** None, or a dictionary, with keys as variables, and their values being what they are substituted as

This is the unification algorithm, which recursively checks to see if there is a substitution of variables for constants or more complicated expressions that will make the two expressions equivalent, if they aren't already.  If no substitution is necessary, an empty dictionary is returned.  If no substitution is possible, None is returned.

In [229]:
def unification( list_expression1, list_expression2):
    if ((is_constant(list_expression1) and is_constant(list_expression2)) or list_expression2==[] or list_expression1==[]):
        if list_expression1==list_expression2:
            return {}
        else:
            return None
    if is_variable(list_expression1):
        if is_inside(list_expression1, list_expression2):
            return None
        elif list_expression1 == list_expression2:
            return {}
        else:
            return {list_expression1:list_expression2}
    if is_variable(list_expression2):
        if is_inside(list_expression2, list_expression1):
            return None
        elif list_expression1 == list_expression2:
            return {}
        else:
            return {list_expression2:list_expression1}
    first1 = list_expression1[0]
    first2 = list_expression2[0]
    result1 = unification(first1,first2)
    if result1 == None:
        return None
    for k in result1.keys():
        for i in range(1,len(list_expression1)):
            list_expression1[i] = denester_replacer(list_expression1[i],k,result1[k])
        for i in range(1,len(list_expression2)):
            list_expression2[i] = denester_replacer(list_expression2[i],k,result1[k])
    result2 = unification(list_expression1[1:],list_expression2[1:])
    if result2 == None:
        return None
    result1.update(result2)
    return result1

In [230]:
def unify( s_expression1, s_expression2):
    return unification( parse( s_expression1), parse( s_expression2))

**Note** If you see the error,

```
tokenize.TokenError: ('EOF in multi-line statement', (2, 0))
```
You most likely have unbalanced parentheses in your s-expression.

## Test Cases

Use the expressions from the Self Check as your test cases...

In [231]:
self_check_test_cases = [
    ['(son Barney Barney)', '(daughter Wilma Pebbles)', None],
    ['(Fred)', '(Barney)', None],
    ['(Pebbles)', '(Pebbles)', {}],
    ['(quarry_worker Fred)', '(quarry_worker ?x)',{'?x':'Fred'}],
    ['(son Barney ?x)','(son ?y Bam_Bam)',{'?y':'Barney','?x':'Bam_Bam'}],
    ['(married ?x ?y)','(married Barney Wilma)',{'?x':'Barney','?y':'Wilma'}],
    ['(son Barney ?x)','(son ?y (son Barney))',{'?y':'Barney','?x':['son', 'Barney']}],
    ['(son Barney ?x)','(son ?y (son ?y))',{'?y':'Barney','?x':['son', 'Barney']}],
    ['(son Barney Bam_Bam)','(son ?y (son Barney))',None],
    ['(loves Fred Fred)','(loves ?x ?x)',{'?x':'Fred'}],
    ['(future George Fred)','(future ?y ?y)',None],
]
for case in self_check_test_cases:
    exp1, exp2, expected = case
    actual = unify(exp1, exp2)
    print(f"actual = {actual}")
    print(f"expected = {expected}")
    print("\n")
    assert expected == actual

actual = None
expected = None


actual = None
expected = None


actual = {}
expected = {}


actual = {'?x': 'Fred'}
expected = {'?x': 'Fred'}


actual = {'?y': 'Barney', '?x': 'Bam_Bam'}
expected = {'?y': 'Barney', '?x': 'Bam_Bam'}


actual = {'?x': 'Barney', '?y': 'Wilma'}
expected = {'?x': 'Barney', '?y': 'Wilma'}


actual = {'?y': 'Barney', '?x': ['son', 'Barney']}
expected = {'?y': 'Barney', '?x': ['son', 'Barney']}


actual = {'?y': 'Barney', '?x': ['son', 'Barney']}
expected = {'?y': 'Barney', '?x': ['son', 'Barney']}


actual = None
expected = None


actual = {'?x': 'Fred'}
expected = {'?x': 'Fred'}


actual = None
expected = None




Now add at least **five (5)** additional test cases of your own making, explaining exactly what you are testing. They should not be testing the same things as the self check test cases above.

In [232]:
new_test_cases = [
    ['(son Barney Barney)', '(daughter Wilma Pebbles)', None, "non-equal constants"],
    
    ['(?x ?y)', '(John (Joe (Joe (Joe ?y))))', None, "explicit nested occurrence check"],
    
    ['(?x ?y)', '(John Joe Joe Joe ?y)', None, "list length difference"],
    
    ['(?x ?y)', '(?x ?y)', {}, "equal variables"],
    
    ['(?x ?y)', '((?y Joe) (?x James))', None, "complicated occurence check"],
    
    ['(?x ?y)', '(?y ?x)', {'?x':'?y'}, "variable swap"],
    
    ['(?x ?x ?x)', '(?y ?z ?w)', {'?x': '?y', '?y': '?z', '?z':'?w'}, "substituting a formula to make it less general"],
    
    ['(?x + ?y = ?z)', '(1 + 1 = 2)', {'?x': '1', '?y': '1', '?z':'2'}, "math should work"],
    
    ['(?x ?z)', '((?y 1) (?w ?x))', {'?x': ['?y', '1'], '?z': ['?w', ['?y', '1']]}, "replacement"]
    
    
    
]
for case in new_test_cases:
    exp1, exp2, expected, message = case
    actual = unify(exp1, exp2)
    print(f"Testing {message}...")
    print(f"actual = {actual}")
    print(f"expected = {expected}")
    print("\n")
    assert expected == actual

Testing non-equal constants...
actual = None
expected = None


Testing explicit nested occurrence check...
actual = None
expected = None


Testing list length difference...
actual = None
expected = None


Testing equal variables...
actual = {}
expected = {}


Testing complicated occurence check...
actual = None
expected = None


Testing variable swap...
actual = {'?x': '?y'}
expected = {'?x': '?y'}


Testing substituting a formula to make it less general...
actual = {'?x': '?y', '?y': '?z', '?z': '?w'}
expected = {'?x': '?y', '?y': '?z', '?z': '?w'}


Testing math should work...
actual = {'?x': '1', '?y': '1', '?z': '2'}
expected = {'?x': '1', '?y': '1', '?z': '2'}


Testing replacement...
actual = {'?x': ['?y', '1'], '?z': ['?w', ['?y', '1']]}
expected = {'?x': ['?y', '1'], '?z': ['?w', ['?y', '1']]}




## Before You Submit...

1. Did you provide output exactly as requested?
2. Did you re-execute the entire notebook? ("Restart Kernel and Rull All Cells...")
3. If you did not complete the assignment or had difficulty please explain what gave you the most difficulty in the Markdown cell below.
4. Did you change the name of the file to `jhed_id.ipynb`?

Do not submit any other files.